In [1]:
import gradio as gr
import pandas as pd
import joblib
import numpy as np

# 1. Load the trained model and original feature names
try:
    model = joblib.load('success_model.pkl')
    original_feature_names = joblib.load('original_feature_names.pkl')
    print("Model and feature names loaded successfully.")
except FileNotFoundError:
    print("Error: Model file or feature names not found. Please run Step 1 (model training and saving) first.")
    # You can exit or provide a default model to continue in development mode
    exit() # Exit the program if files are not found

# 2. Define the prediction function (This is the function that Gradio will call)
def predict_loan_approval(freetime, past_failures, g1_score):
    # Create a DataFrame from user inputs
    # Columns must be in the same order and have the same names as the original pre-processed columns
    input_data = pd.DataFrame([[freetime, past_failures, g1_score]],
                              columns=['freetime', 'failures', 'G1'])

    # Ensure all original columns are present, and fill missing columns with default values if necessary
    # (This is important if your model relies on columns not directly entered by the user)
    # In our case, input_data contains all the columns used to train the model, so this part may not be strictly necessary.
    # However, if X_train contained other columns (e.g., One-Hot encoded columns from features not in the direct input)
    # then this step would be crucial to ensure input_data has the same structure as X_train
    # before passing it to the pipeline.

    # Example of how to add missing columns if needed (uncomment and adapt if your model expects more)
    # for col in original_feature_names:
    #     if col not in input_data.columns:
    #         input_data[col] = 0 # Or any suitable default value for that feature

    # Use the pipeline to predict (it will perform pre-processing then predict)
    prediction_proba = model.predict_proba(input_data)[0] # Probability of each class
    prediction_class = model.predict(input_data)[0] # Predicted class (0 or 1)

    # 3. Formulate the prediction and explanation
    if prediction_class == 1:
        result = "Student will succeed"
        explanation = f"The model predicts Student success or failure  with a probability of {prediction_proba[1]*100:.2f}%."
        explanation += "\nBased on the provided factors, you seem to have a good chance of success."
    else:
        result = "Student will fail"
        explanation = f"The model predicts loan non-approval with a probability of {prediction_proba[0]*100:.2f}%."
        explanation += "\nSome factors may reduce the chance of success."

    # To enhance the explanation, we can try to interpret the effect of some key features
    # This step ideally requires libraries like SHAP or LIME for complex model explanations.
    # But here, we can provide a simple, rule-based interpretation for illustrative purposes.
    if freetime < 2:
        explanation += "\nObservation: Low Free time might have a negative impact."
    if past_failures > 0:
        explanation += "\nObservation: Previous failures might have a negative impact."
    if g1_score < 10:
        explanation += "\nObservation: Lower previous grades might affect success chances."

    return result, explanation

# 4. Build the Gradio interface
iface = gr.Interface(
    fn=predict_loan_approval,
    inputs=[
        gr.Slider(minimum=0, maximum=5, step=1, label="Free Time (hours per week)", value=2),
        gr.Slider(minimum=0, maximum=3, step=1, label="Previous Failures", value=0),
        gr.Slider(minimum=0, maximum=20, step=1, label="Previous Grade G1", value=10),
    ],
    outputs=[
        gr.Textbox(label="Prediction Result"),
        gr.Textbox(label="Explanation of Prediction")
    ],
    title="Predicting student success or failure",
    description="Enter student data to predict whether or not they will pass, along with an explanation of the prediction.",
    flagging_mode="auto" # Allows users to flag results (optional)
)

# 5. Launch the app
iface.launch(share=True) # share=True to create a temporary public link

Model and feature names loaded successfully.
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://40b9fb8ea9095d677f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
